In [ ]:
#make IPython notebook matplotlib plot inline
%matplotlib inline 

#import modules and libraries
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from pathlib import Path
from PIL import Image
from scipy import ndimage
from skimage import measure
from skimage.io import imread,imsave

INPUT_DIR = '/mnt/med-groups-lmu/ls1/users/j/jilehtim/HARRI/PYTHON_INPUT'
OUTPUT_DIR = '/mnt/med-groups-lmu/ls1/users/j/jilehtim/HARRI/PYTHON_OUTPUT'

INPUT_DIR = Path(INPUT_DIR)
if not INPUT_DIR.is_dir():
    raise ValueError('Please check INPUT_DIR.')

OUTPUT_DIR = Path(OUTPUT_DIR)
if not OUTPUT_DIR.is_dir():
    raise ValueError('Please check OUTPUT_DIR.')

MIN_FRAMES = 4


In [ ]:

def process(file):
    print(str(file))
    im = imread(str(file), plugin="tifffile")
    print(im.shape)
    print(im.dtype)

    # select white pixels into blobs
    blobs = im == 255
    # label blobs
    labels = measure.label(blobs)
    # measure labeled blobs
    props = measure.regionprops_table(labels)
    # read measurements into a dataframe
    df = pd.DataFrame(props)
    # add column for and calculate blink duration
    df['duration'] = df['bbox-3'] - df['bbox-0']

    print(df.shape)
    print(df.head())

    # filter out short blinks
    df = df[df['duration'] >= MIN_FRAMES]
    print(df.shape)
    long_blinks = df.label.values
    print(long_blinks)

    # set label to zero in labels image if the label is not in long_blinks
    ul = np.unique(labels)
    for l in ul:
        if l not in long_blinks:
            labels[labels==l] = 0

    # save modified labels image
    from skimage import img_as_uint
    labels = img_as_uint(labels)
    output = OUTPUT_DIR / (file.stem + '_long_blinks.ome.tif')
    print(str(output))
    imsave(str(output),labels)

    # save measurements
    output = OUTPUT_DIR / (file.stem + '_long_blinks.csv')
    print(str(output))
    df.to_csv(output, index=False)
    
    return labels
    
    
# process all .ome.tif files in input directory    
files = INPUT_DIR.glob('*.ome.tif')
labels = None
for f in files:
    file = INPUT_DIR / f
    if not file.exists():
        raise ValueError('Please check file name: ' + str(file))
    labels = process(f)

   
    
    

In [ ]:
z=35
fig, axs = plt.subplots(1, 2, figsize=(15, 10))
axs[0].imshow(labels[z],cmap='coolwarm')
axs[1].imshow(labels[z],cmap='coolwarm')

In [ ]:
help(imsave)